In [7]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import os
import sys

import torch
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset

from EduCDM import GDIRT


In [2]:
sys.path.append(os.path.abspath('..'))

In [3]:
from utils.dataset import LogDataSet

In [4]:
SEED = 2

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
PROJECT_ROOT = os.path.split(os.getcwd())[0]
DATA_PRO = 'data/processed'
QC23_log_pro = 'qc23_log_pro'
CSV = '.csv'
df = pd.read_csv(os.path.join(PROJECT_ROOT, DATA_PRO, QC23_log_pro + CSV))

In [101]:
df = df[df.correctness.isin([0.0,1.0])].reset_index(drop=True)

In [102]:
df.head()

,problem_log_id,student_user_id,problem_id,correctness
0,15444,523,1601582,1.0
1,15445,566,1601584,1.0
2,15446,505,1601582,1.0
3,15447,509,1601582,1.0
4,15449,550,1601584,1.0


In [103]:
data = LogDataSet(df)

In [104]:
len(data)

14548521

In [105]:
len(data), len(data.user_id2idx), len(data.item_id2idx)

(14548521, 436537, 38364)

In [ ]:
batchsize = 512
trainsize = 0.8
testsize = 0.1

def transform(dataset, batch_size, train_size, test_size, **params):
    eval_size = 1 - train_size - test_size
    train_dataset, test_dataset, eval_dataset = random_split(dataset, [train_size, test_size, eval_size])
    return DataLoader(train_dataset, batch_size, **params), DataLoader(eval_dataset, batch_size, **params), DataLoader(test_dataset, batch_size, **params)

In [107]:
train, eval, test = transform(data, batchsize, trainsize, testsize, shuffle=True)
train, eval, test

(<torch.utils.data.dataloader.DataLoader at 0x7f55ac752f90>,
 <torch.utils.data.dataloader.DataLoader at 0x7f56286dab70>)

In [108]:
len(train.dataset), len(eval.dataset), len(test.dataset)

(11638817, 1454852, 1454852)

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [114]:
cdm = GDIRT(len(data.user_id2idx), len(data.item_id2idx))

cdm.train(train, eval, epoch=2, device = device, lr = 0.010)
cdm.save("irt.params")

Epoch 0:   0%|          | 0/45465 [00:00<?, ?it/s]

Epoch 0:   4%|▍         | 1927/45465 [00:45<17:03, 42.55it/s]  


KeyboardInterrupt: 